<a href="https://colab.research.google.com/github/alpharomeo7/rob_mnist/blob/master/MNIST_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [3]:
from pathlib import Path
import requests
import pickle
import gzip
def load_data(): 
  DATA_PATH = Path("data")
  PATH = DATA_PATH / "mnist"

  PATH.mkdir(parents=True, exist_ok=True)

  URL = "https://github.com/pytorch/tutorials/raw/master/_static/"
  FILENAME = "mnist.pkl.gz"

  if not (PATH / FILENAME).exists():
          content = requests.get(URL + FILENAME).content
          (PATH / FILENAME).open("wb").write(content)

  with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
          return pickle.load(f, encoding="latin-1")


((x_train, y_train), (x_valid, y_valid), (x_test, y_test)) = load_data()

In [4]:
import numpy as np
convx_train =  np.reshape(x_train, (x_train.shape[0], 28,28))
convx_test =  np.reshape(x_test, (x_test.shape[0], 28,28))

convx_train = convx_train[..., tf.newaxis].astype("float32")
convx_test = convx_test[..., tf.newaxis].astype("float32")

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (convx_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((convx_test, y_test)).batch(32)

In [6]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.14551937580108643, Accuracy: 95.59400177001953, Test Loss: 0.07758539915084839, Test Accuracy: 97.5999984741211
Epoch 2, Loss: 0.04224982112646103, Accuracy: 98.66200256347656, Test Loss: 0.05657080188393593, Test Accuracy: 98.13999938964844
Epoch 3, Loss: 0.020944489166140556, Accuracy: 99.35399627685547, Test Loss: 0.056291673332452774, Test Accuracy: 98.20999908447266
Epoch 4, Loss: 0.013229665346443653, Accuracy: 99.57200622558594, Test Loss: 0.06472153961658478, Test Accuracy: 98.38999938964844
Epoch 5, Loss: 0.010260367766022682, Accuracy: 99.66400146484375, Test Loss: 0.0811733528971672, Test Accuracy: 97.90999603271484


In [12]:
def reshape_conv_data(x): 
  x = np.reshape(x, (x.shape[0], 28,28))
  return x[..., tf.newaxis].astype("float32") 

def predict(x_test): 
  x_test = reshape_conv_data(x_test)
  probabilities = model(x_test, training=False)
  predictions = [np.argmax(prediction) for prediction in probabilities]
  return predictions

In [13]:
#Reload data 
def load_data(): 
  DATA_PATH = Path("data")
  PATH = DATA_PATH / "mnist"

  PATH.mkdir(parents=True, exist_ok=True)

  URL = "https://github.com/pytorch/tutorials/raw/master/_static/"
  FILENAME = "mnist.pkl.gz"

  if not (PATH / FILENAME).exists():
          content = requests.get(URL + FILENAME).content
          (PATH / FILENAME).open("wb").write(content)

  with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
          return pickle.load(f, encoding="latin-1")


((x_train, y_train), (x_valid, y_valid), (x_test, y_test)) = load_data()

In [16]:
np.sum(predict(x_test) == y_test)/len(y_test)

0.9791